# 修了課題DEMO③ 仮想通貨

[JDLAが策定しているバージョン](https://www.jdla.org/certificate/engineer/)に合わせるために、以下のセルの実行をお願いします．

（#コメントアウト されているものは必要ありません）

また実行完了後に「ランタイムの再起動」をして下さい．

（以下のセルの実行は、最初にしていただければ、以降必要ありません．）

In [ ]:
%%capture
!pip uninstall matplotlib -y
!pip install matplotlib==3.7.1

# !pip uninstall opencv-python -y
# !pip install opencv-python==4.7.0.72

# !pip uninstall torch -y
# !pip install torch==2.0.1

# !pip uninstall torchvision -y
# !pip install torchvision==0.15.2

##はじめに
この修了課題では、仮想通貨の情報を利用して、

未来の仮想通貨の価格予想を行うものです。

今回はDEMOとして、RNN(再帰型ニューラルネットワーク)を用いて学習をさせてみましょう。

##作成までの流れ
大まかな流れとして
1. データのダウンロードと成形

   データダウンロードしてそのまま活用するのは困難です。
   モデルが学習できるよう、カテゴリ変数に置き換えたり、
   欠損値を補完したりなど、成形する必要があります。
2. モデルの構築

   学習を行うモデルのアルゴリズムを理解して、コードを作成します。

3. 学習と結果

   学習を行い結果を確認してみます。
   精度が目標まで達したら、提出用のデータを作成します。

#1.データのダウンロードと成形

## データのダウンロード

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 学習データのダウンロード
!wget 'https://drive.google.com/uc?export=download&id=1kUfPb8qikA8rdQ26iVUxpod2Qjw3ct_O' -O crypto_train.csv

# テストデータのダウンロード
!wget 'https://drive.google.com/uc?export=download&id=1VhzCcjNSDxGRG86Za653zHHpjVCdSPD3' -O crypto_test_x.csv

In [ ]:
# 学習データの確認
train = pd.read_csv('crypto_train.csv', index_col=0)
train

## データの成形

今回使用するRNNのために必要な処理を施していきます。

In [ ]:
# 正規化処理
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler = scaler.fit(train.values)

# 正規化する
s_train = scaler.transform(train)
# 元の値に戻す場合
inv_train = scaler.inverse_transform(s_train)

In [ ]:
# データの形状確認
print(s_train.shape)

In [ ]:
# シーケンスデータの教師ありデータを作成する
def make_seq(data, seq_len=6):
    X = data[:, 0:seq_len]
    X = np.expand_dims(X, axis=2)
    Y = data[:, seq_len:]
    return X, Y

X, Y = make_seq(s_train)

def train_val_split(X, Y, val_rate):
  rate = int(X.shape[0]*(1-val_rate))
  train_X, train_Y, val_X, val_Y = X[:rate], Y[:rate], X[rate:], Y[rate:]
  return train_X, train_Y, val_X, val_Y

train_X, train_Y, val_X, val_Y = train_val_split(X, Y, 0.1)

In [ ]:
# データの形状確認
print('train_X:', train_X.shape)
print('train_Y:', train_Y.shape)

# 2.モデルの構築

## モデル容量の設定

In [ ]:
# 入力データのひとつあたりの次元
input_size = 1
# モデル容量の定義
hidden_dim = 10
# 出力データのひとつあたりの次元
output_dim = input_size
U = (np.random.randn(input_size, hidden_dim)/np.sqrt(hidden_dim)).astype('f')
W = (np.random.randn(hidden_dim, hidden_dim)/np.sqrt(hidden_dim)).astype('f')
b = np.zeros(hidden_dim).astype('f')
V = (np.random.randn(hidden_dim, output_dim)/np.sqrt(hidden_dim)).astype('f')
c = np.zeros(output_dim).astype('f')

#3.精度確認から提出まで

##学習

In [ ]:
# 学習のハイパーパラメータ
epochs = 1000
learning_rate = 0.01

# 勾配爆発を防ぐために勾配クリッピングを行う
min_clip_value = -10
max_clip_value = 10

# データの設定
X = train_X
Y = train_Y

# 勾配クリッピングの関数定義
def clip_gradient(matrix):
    if np.any(matrix > max_clip_value):
        matrix[matrix > max_clip_value] = max_clip_value
    if np.any(matrix < min_clip_value):
        matrix[matrix < min_clip_value] = min_clip_value
    return matrix

# 学習
for epoch in range(epochs):
    batch_size = X.shape[0]
    prev_ss = {}
    prev_s = np.zeros((batch_size, hidden_dim))

    # 順伝播
    for i in range(X.shape[1]):
        new_input = X[:,  i]
        mulu = np.dot(new_input, U)
        mulw = np.dot(prev_s, W)
        add = mulw + mulu + b
        prev_s = np.tanh(add)
        prev_ss[str(i)] = prev_s

    # 予測と損失計算
    pred = np.dot(prev_s, V) + c
    per_loss = (Y-pred)**2/2
    loss = np.sum(per_loss)/(batch_size)
    print('epoch:', epoch+1, 'Loss:', loss)

    # 逆伝播
    e_o = pred-Y

    dV = np.dot(pred.T, e_o)
    dc = np.sum(e_o)/batch_size

    dU = 0
    dW = 0
    db = 0

    for t in range(X.shape[1]):
        e_h = (1 - prev_s[t]**2) * np.dot(e_o, V.T)
        new_input = X[:,  i]
        # new_input = np.expand_dims(X[:,  i], axis=1)

        if t==0:
          dU = np.dot( new_input.T, e_h)
          dW = np.dot(np.zeros((batch_size, hidden_dim)).T, e_h)
          db = np.sum(e_h, axis=0)
        else:
          dU += np.dot(new_input.T, e_h)
          dW += np.dot(prev_ss[str(t-1)].T, e_h)
          db += np.sum(e_h, axis=0)/batch_size

    # 勾配クリッピングの適用
    dU = clip_gradient(dU)
    dV = clip_gradient(dV)
    dW = clip_gradient(dW)

    # 重みを更新
    U -= learning_rate * dU
    V -= learning_rate * dV
    W -= learning_rate * dW
    b -= learning_rate * db
    c -= learning_rate * dc

In [ ]:
# 検証データで精度を確認する
X = val_X
Y = val_Y

# 検証データの予測
batch_size = X.shape[0]
prev_s = np.zeros((batch_size, hidden_dim))

# ネットワークを通して順伝播
for i in range(X.shape[1]):
    new_input = X[:,  i]
    mulu = np.dot(new_input, U)
    mulw = np.dot(prev_s, W)
    add = mulw + mulu + b
    prev_s = np.tanh(add)
    prev_ss[str(i)] = prev_s

# 出力層からの予測値
pred = np.dot(prev_s, V) + c

# スケーラーを使用して、標準化されたデータを元のスケールに戻す
scaler = scaler.fit(train.values.reshape(-1, 1))
Y = scaler.inverse_transform(Y)
pred = scaler.inverse_transform(pred)

# sklearnのmean_squared_error関数を使用してRMSEを計算
from sklearn.metrics import mean_squared_error
import math

# 平均二乗誤差の平方根（RMSE）を計算
rmse = math.sqrt(mean_squared_error(Y, pred))
print('検証RMSE:', rmse)

In [ ]:
plt.plot(Y[:])
plt.plot(pred[:])

今回の検証RMSEの値を確認すると、このモデルの仕様では目標の精度には達していないことが分かります。

他のライブラリをインポートして、より精度が高くなるようなモデルを構築してみると

精度が向上するかもしれません。

## テストデータに対して予測値を出力する
今回学習したRNNが

テスト用のデータを予測して、結果をcsvファイルとして出力するまでを

掲載してみました。

このcsvファイルを「修了課題提出用サイト」にアップロードすると結果を確認することができます。

In [ ]:
# テストデータの確認
test_x = pd.read_csv('crypto_test_x.csv', index_col=0)
test_x

In [ ]:
# テストデータも正規化する
scaler = scaler.fit(test_x)
s_test_x = scaler.transform(test_x)

# データ形式をモデルに対して適正化する
s_test_x = np.expand_dims(s_test_x, axis=2)

# 検証データで精度を確認する
X = s_test_x

# 検証データの予測
batch_size = X.shape[0]
prev_s = np.zeros((batch_size, hidden_dim))
for i in range(X.shape[1]):
    new_input = X[:,  i]
    mulu = np.dot(new_input, U)
    mulw = np.dot(prev_s, W)
    add = mulw + mulu + b
    prev_s = np.tanh(add)
    prev_ss[str(i)] = prev_s
pred = np.dot(prev_s, V) + c

# もとに戻した値
scaler = scaler.fit(test_x.values.reshape(-1, 1))
inv_pred = scaler.inverse_transform(pred)

In [ ]:
# 予測した値を出力する
pred = pd.DataFrame(inv_pred, columns=['Sun'])
pred.to_csv('crypto_pred.csv')
pred